### Import library

In [1]:
import pandas as pd
from bokeh.io import output_notebook,show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource,HoverTool,DatetimeTickFormatter
from bokeh.models.annotations import Arrow,Label,BoxAnnotation
from bokeh.models.arrow_heads import VeeHead

In [2]:
output_notebook()

Loading BokehJS ...

### Read dataset

In [4]:
df=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,64575,65080,65486,65728,66275,66903,67743,68366,69130,70111
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132118,132153,132176,132209,132215,132229,132244,132264,132285,132297
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,126156,126434,126651,126860,127107,127361,127646,127926,128198,128456
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,13569,13569,13569,13569,13569,13664,13671,13682,13693,13693
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,31661,31909,32149,32441,32623,32933,33338,33607,33944,34180


In [5]:
df.shape

(276, 498)

### Wide to long dataframe

In [6]:
df_melt=pd.melt(df, id_vars=['Province/State', 'Country/Region', 'Lat', 
                             'Long'], var_name='date', 
                value_name='confirmed_case')
df_melt.head()

,Province/State,Country/Region,Lat,Long,date,confirmed_case
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [7]:
df_melt.shape

(136344, 6)

### Filter bd dataframe

In [8]:
df_bd=df_melt[df_melt["Country/Region"]=='Bangladesh']
bd=df_bd.loc[:, ["date", "confirmed_case"]]
bd['daily_case']=df_bd.confirmed_case.diff().fillna(0)

In [9]:
bd.head()

,date,confirmed_case,daily_case
20,1/22/20,0,0.0
296,1/23/20,0,0.0
572,1/24/20,0,0.0
848,1/25/20,0,0.0
1124,1/26/20,0,0.0


In [10]:
bd.tail()

,date,confirmed_case,daily_case
134984,5/25/21,792196,1675.0
135260,5/26/21,793693,1497.0
135536,5/27/21,794985,1292.0
135812,5/28/21,796343,1358.0
136088,5/29/21,797386,1043.0


In [11]:
bd.shape

(494, 3)

In [12]:
bd.dtypes

date               object
confirmed_case      int64
daily_case        float64
dtype: object

### Convert date column to pandas datetime object

In [13]:
bd["date"]=pd.to_datetime(bd.date)

In [14]:
bd.dtypes

date              datetime64[ns]
confirmed_case             int64
daily_case               float64
dtype: object

### Pandas `DataFrame` to Bokeh `ColumnDataSource`

In [15]:
bd_source=ColumnDataSource(bd)

### Interactive plot

In [16]:
hover=HoverTool(tooltips=[("date", "@date{%F}"), 
                    ("daily case", "@daily_case")], 
formatters={"@date" : "datetime"}, mode="mouse")
            

p=figure(plot_width=600, plot_height=400, x_axis_type="datetime",
        tools=[hover, "pan", "reset", "save"], 
        title="Daily Covid-19 cases in Bangladesh", 
        sizing_mode="stretch_both")
#p.add_tools(hover)
p.line(x="date", y="daily_case", source=bd_source, 
       color="DarkBlue", 
       line_width=1.5)

# add arrow
arrow1=Arrow(x_start=pd.to_datetime("2020-03-08"),
             y_start=1000,
             x_end=pd.to_datetime("2020-03-08"), 
             y_end=3, 
             line_color="red",
             end=VeeHead(size=8,line_color="red", fill_color="red"))
p.add_layout(arrow1)

#add frist case label
label1=Label(x=pd.to_datetime("2020-01-03"),
        y=1000, 
        text_baseline="middle",
        text="First Covid-19 Cases 8 March", 
        text_color="red")
p.add_layout(label1)

#add first lockdown box
first_lockdown=BoxAnnotation(left=pd.to_datetime("2020-03-23"), 
                            right=pd.to_datetime("2020-05-30"), 
                            top=8000, 
                            bottom=0, 
                            fill_color="gray",
                            fill_alpha=0.3)
p.add_layout(first_lockdown)

lockdown_label1=Label(x=pd.to_datetime("2020-03-23"), 
                     y=6000, 
                     text="Lockdown", 
                     text_color="gray")
p.add_layout(lockdown_label1)
 
#add second lockdown annotatino
second_lockdown=BoxAnnotation(left=pd.to_datetime("2021-04-14"),
                             top=8000, 
                             bottom=0, 
                             fill_color="gray", 
                             fill_alpha=0.3)
p.add_layout(second_lockdown)
lockdown_label2=Label(x=pd.to_datetime("2021-04-14"), 
                     y=6000, 
                     text="Lockdown", 
                     text_color="gray")
p.add_layout(lockdown_label2)


p.xaxis.axis_label="Time"
p.xaxis.axis_label_text_color="DarkGreen"
p.xaxis.formatter=DatetimeTickFormatter(months="%b %y")

p.yaxis.axis_label="Daily infection"
p.yaxis.axis_label_text_color="DarkGreen"

p.title.text_font="ecotype"
p.title.text_color="DarkGreen"
p.title.text_font_size="20px"
show(p)